# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [119]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [120]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [121]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [122]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [267]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()
root.tag

'mondial'

# Answer 1

In [728]:
# Dictionary with key as Country name and value as Infant Mortality Name
infant_mortality_dict={}
for element in document.iter('country'):
    try:
        infant_mortality_dict[element.find('name').text]=float(element.find('infant_mortality').text)
    except:
        pass
df_infant_mortality = pd.DataFrame(data=infant_mortality_dict.items(),columns=['Name','Infant_Moratlity'])
df_infant_mortality.sort_values(by='Infant_Moratlity',inplace=True)
df_infant_mortality.head(10)

,Name,Infant_Moratlity
34,Monaco,1.81
210,Japan,2.13
71,Norway,2.48
64,Bermuda,2.48
76,Singapore,2.53
106,Sweden,2.60
55,Czech Republic,2.63
143,Hong Kong,2.73
52,Macao,3.13
189,Iceland,3.15


# Answer 2

In [729]:
# A list which has dictionary as its objects.Dictionry has city,year and population as its attributes
li_cities=[]
for country in document.iterfind('country'):
    for city in country.iter('city'):
        cityname = city.find('name').text
        #li_cities.append(city)
        
        for p in city.iter('population'):
            city_pop={}
            city_pop['city']=cityname
            city_pop['year']=p.attrib['year']
            city_pop['population']=int(p.text)
            li_cities.append(city_pop)
        
        
#Using group by city to get population of latest year
df_city_population = pd.DataFrame(li_cities)
idx = df_city_population.groupby(['city'])['year'].transform(max) == df_city_population['year']
df_CityPopulation=df_city_population[idx]
df_CityPopulation =pd.DataFrame(df_CityPopulation)
df_CityPopulation.sort_values(by='population',ascending = False,inplace=True)
#Top ten cities with largest population
df_CityPopulation.head(10)

,city,population,year
3750,Shanghai,22315474,2010
2607,Istanbul,13710512,2012
4303,Mumbai,12442373,2011
1546,Moskva,11979529,2013
3746,Beijing,11716620,2010
8208,São Paulo,11152344,2010
3754,Tianjin,11090314,2010
3364,Guangzhou,11071424,2010
4399,Delhi,11034555,2011
3371,Shenzhen,10358381,2010


# Answer 3

In [730]:
import numpy as np
import pandas as pd
li_main=[]
for element in document.iter('country'):
    li_sub =[]
    for ethnicgroup in element.iter('ethnicgroup'):
        
        try:
            li_sub.append((element.find('name').text,ethnicgroup.text,float(ethnicgroup.attrib['percentage'])))
        except:
            pass
    li_main.append(li_sub)
#A list which comprises of tuples with name of city,ethnic group name and ethnic group percentage.
li_final=[j for i in li_main for j in i]


df_EthnicGroups = pd.DataFrame(li_final,columns=['CountryName','Group','Percentage'])

li_population=[]
for country in document.iterfind('country'):
    country_name = country.find('name').text
    for e in country.findall('population'):
        pop_dict ={}
        pop_dict['population'] =int(e.text)
        pop_dict['year'] = e.attrib.get('year', np.nan)
        pop_dict['CountryName'] = country_name
        li_population.append(pop_dict)
#A list comprising dictionary as its object with countryname,year and population as its attributes.
df_CountryPopulation = pd.DataFrame(li_population)
#Using group by country to get population of latest year
idx = df_CountryPopulation.groupby(['CountryName'])['year'].transform(max) == df_CountryPopulation['year']
df_CountryPopulation_latest_year=df_CountryPopulation[idx]
#Merging the two DataFrames on CountryName
df_question3 = df_EthnicGroups.merge(df_CountryPopulation_latest_year,how='left',on='CountryName')
df_question3.drop('year',axis=1,inplace=True)
#Calculating the percentage of ethnic group
df_question3['percent_group']=((df_question3['Percentage']/100)*df_question3['population']).astype(int)
ethnicgroups = df_question3.groupby('Group').sum(by='percent_group')
ethnicgroups.sort_values(by='percent_group',ascending=False,inplace=True)
ethnicgroups.drop('CountryName',axis=1,inplace =True)
ethnicgroups.drop('Percentage',axis=1,inplace =True)
ethnicgroups.head(10)

,population,percent_group
Group,,
Han Chinese,1360720000,1245058800
Indo-Aryan,1210854977,871815583
European,1157295639,494872201
African,975352746,318325104
Dravidian,1210854977,302713744
Mestizo,279743964,157734349
Bengali,149772364,146776916
Russian,322438406,131856989
Japanese,127298000,126534212


# Answer 4a

In [731]:
river_list=[]
for river in document.iter('river'):
    try:
        river_list.append((river.attrib['country'],river.find('name').text,int(river.find('length').text)))
    except:
        pass
#river_list[:4]
df_river = pd.DataFrame(data=river_list,columns=['Country_Code','Name','Length'])
df_river.sort_values(by='Length',ascending = False,inplace = True)
df_longest =df_river.head(10)

d=df_longest.T.to_dict().values()

li_final=[]
for i in d:
    #print(i['Country_Code'])
    j = i['Country_Code'].split()
    #li_final.append(i)
    if (len(j)==1):
        li_final.append(i)
    if (len(j) > 1):
        for k in range(len(j)):
            curr ={}
            curr['Country_Code'] = j[k]
            curr['Length']=i['Length']
            curr['Name']=i['Name']
            li_final.append(curr)

df_final = pd.DataFrame(li_final)
df_river_merge = df_final.merge(df_country_code,on='Country_Code')   
df_river_merge.sort_values(by='Length',ascending = False,inplace = True)
df_river_merge.head(10)

,Country_Code,Length,Name,CName
8,CO,6448,Amazonas,Colombia
10,PE,6448,Amazonas,Peru
9,BR,6448,Amazonas,Brazil
1,CN,6380,Jangtse,China
0,CN,4845,Hwangho,China
13,R,4400,Lena,Russia
20,RCB,4374,Zaire,Congo
21,ZRE,4374,Zaire,Zaire
7,VN,4350,Mekong,Vietnam
6,K,4350,Mekong,Cambodia


# Answer 4b

In [732]:
lake_list =[]
for lake in document.iter('lake'):
    try:
        lake_list.append((lake.attrib['country'],lake.find('name').text,int(lake.find('area').text)))
    except:
        pass
#lake_list
df_lake = pd.DataFrame(data=lake_list,columns=['Country_Code','LName','Area'])
df_lake.sort_values(by='Area',ascending = False,inplace = True)
df_longestlake =df_lake.head(10)
d=df_longestlake.T.to_dict().values()
li_final_lake=[]
for i in d:
    #print(i['Country_Code'])
    j = i['Country_Code'].split()
    #li_final.append(i)
    if (len(j)==1):
        li_final_lake.append(i)
    if (len(j) > 1):
        for k in range(len(j)):
            curr ={}
            curr['Country_Code'] = j[k]
            curr['Area']=i['Area']
            curr['LName']=i['LName']
            li_final_lake.append(curr)
#li_final_lake
df_final_lake = pd.DataFrame(li_final_lake)
df_lake_merge = df_final_lake.merge(df_country_code,on='Country_Code')
df_lake_merge.sort_values(by='Area',ascending=False,inplace=True)
df_lake_merge.head(10)

,Area,Country_Code,LName,CName
24,386400,TM,Caspian Sea,Turkmenistan
7,386400,R,Caspian Sea,Russia
23,386400,IR,Caspian Sea,Iran
22,386400,KAZ,Caspian Sea,Kazakhstan
21,386400,AZ,Caspian Sea,Azerbaijan
0,82103,CDN,Lake Superior,Canada
3,82103,USA,Lake Superior,United States
12,68870,EAU,Lake Victoria,Uganda
8,68870,EAT,Lake Victoria,Tanzania
11,68870,EAK,Lake Victoria,Kenya


# Answer 4c

In [733]:
airport_list =[]
for airport in document.iter('airport'):
    try:
        airport_list.append((airport.attrib['country'],airport.find('name').text,int(airport.find('elevation').text)))
    except:
        pass
df_airport = pd.DataFrame(data=airport_list,columns=['Country_Code','AName','AElevation'])
df_airport.sort_values(by='AElevation',ascending = False,inplace = True)
df_airport =df_airport.head(10)
df_airport_merge=df_airport.merge(df_country_code,on='Country_Code')
df_airport_merge.sort_values(by='AElevation',ascending=False,inplace=True)
df_airport_merge.head(1)

,Country_Code,AName,AElevation,CName
0,BOL,El Alto Intl,4063,Bolivia
